<a href="https://colab.research.google.com/github/frusu2005/AIRemote06/blob/master/homework01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training and Testing an MNIST Classifier in Google Colab

In [ ]:
# --- STEP 1: Imports and Data ---
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize and reshape
#CNNs expect a 4D tensor: (samples, height, width, channels) — 1 channel because grayscale.
#astype('float32') / 255: Normalizes pixel values from [0,255] → [0,1].
X_train = X_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
X_test  = X_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

# --- STEP 2: Model Definition ---
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)), #Applies 32 small 3×3 filters (kernels) to the image. Each filter learns to detect patterns like edges or corners. Output: 32 feature maps.
                                                                 #ReLU = Rectified Linear Unit = max(0, x). Makes the network nonlinear so it can learn complex patterns.
    MaxPooling2D((2,2)),         #Downsamples the feature maps by taking the maximum value in 2×2 windows. Reduces size, computation, and adds translation invariance.
    Conv2D(64, (3,3), activation='relu'),    # Adds a deeper convolutional layer with 64 filters. Now learns more abstract features (e.g., loops, shapes).
    MaxPooling2D((2,2)),       #Reduces size again — so now we’re summarizing what was learned.

    Flatten(),      #Converts the 2D feature maps into a 1D vector. Prepares data for the dense (fully connected) layers.
    Dense(64, activation='relu'),  #Fully connected layer with 64 neurons. Combines all the learned features to make final decisions
    Dense(10, activation='softmax')   #Output layer: 10 neurons (for digits 0–9). softmax converts outputs to probabilities that sum to 1.
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])   # model compilation

# --- STEP 3: Train ---
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1, callbacks=[early_stop])

# --- STEP 4: Evaluate ---
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Save model for later
model.save('mnist_model.h5')


Test With Uploaded Picture

In [ ]:
import cv2
from google.colab import files

# Upload image
uploaded = files.upload()  # choose your file manually

# Load model
model = load_model('mnist_model.h5')

# Load and preprocess image
img = cv2.imread('my_digit.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))
img = cv2.bitwise_not(img)         # invert if white background
img = img.astype('float32') / 255.0
img = img.reshape(1, 28, 28, 1)

# Predict
pred = model.predict(img)
digit = np.argmax(pred)

print("Predicted digit:", digit)

# Show the image
plt.imshow(img.reshape(28,28), cmap='gray')
plt.title(f"Predicted: {digit}")
plt.axis('off')
plt.show()
